In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from konlpy.tag import Mecab
from tqdm import tqdm
import re

import collections
import seaborn as sns

In [2]:
#한글 깨짐 처리
import os

if os.name == 'posix':
    plt.rc("font", family="AppleGothic")

else :
    plt.rc("font", family="Malgun Gothic")

%config InlineBackend.figure_format = 'retina'

In [3]:
train = pd.read_csv("capstone_dataset/train.csv")

In [4]:
train.columns

Index(['id', 'title', 'body', 'category', 'label'], dtype='object')

In [16]:
true_df = train[train['label']==0]

In [17]:
true_df.category.value_counts()

7    12903
6    10955
2     8875
3     8240
1     5981
4     2887
5      145
0       11
8        3
Name: category, dtype: int64

In [18]:
len(true_df)

50000

- category : 0~8
- 5,0,8은 다 사용하고, 나머지는 앞에 1000개 target 사용하기/ 뒤에 1000개 sample용
- target 3부분으로 나누기. 30%, 70%지점으로 끊기
- t1,t2,t3/s1,s2,s3
- type A : t1,t2,/s2,s3/,t3
- type B :t1,sample dataset, t3
- type C : t1, /s2,s3
- type D : s1,/ t2/, s3

category 9개 * type 4개 => 1000개 데이터 * 각 

In [15]:
train.head(1)

,id,title,body,category,label
0,AKR20191006026700004,공소시효 만료로 수배 해제 5년간 2만3천건…하루 13건꼴,공소시효가 만료돼 수배가 해제되는 사건이 하루 12∼13건꼴인 것으로 조사됐다. 6...,3,0


In [61]:
df1 = true_df[true_df['category']==3].reset_index(drop=True)
df1.head()

,id,title,body,category,label
0,AKR20191006026700004,공소시효 만료로 수배 해제 5년간 2만3천건…하루 13건꼴,공소시효가 만료돼 수배가 해제되는 사건이 하루 12∼13건꼴인 것으로 조사됐다. 6...,3,0
1,AKR20190201116900051,설 연휴 부산 병·의원 2천75곳 문 연다,이번 설 연휴 부산에서는 모두 2천75곳의 병·의원이 문을 연다. 부산시는 2일부터...,3,0
2,AKR20190829062751054,5·18사적지 옛 적십자병원 매각…광주시 매입 난색,5·18민주화운동 사적지 제11호인 옛 광주적십자병원 건물이 매각될 것으로 알려지면...,3,0
3,AKR20190522007500004,최고기온 30도 내외로 더워…수도권 오전에 미세먼지 나쁨,수요일인 22일은 전국이 맑고 내륙지역은 낮 최고기온이 30도 내외로 덥겠다. 이날...,3,0
4,AKR20190524026451004,"미소페 제화공들 본사 앞 점거 농성…""고용 보장하라""","구두 브랜드인 미소페 공장에서 일하는 제화노동자들이 미소페 본사 앞을 점거하고 ""본...",3,0


In [30]:
s = sample.split('.')
len_s = len(s)
int(len_s * 0.3), int(len_s * 0.7), len_s

(2, 4, 7)

In [48]:
category = [i for i in range(9)]

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [77]:
target_df.loc[0,'title']

'공소시효 만료로 수배 해제 5년간 2만3천건…하루 13건꼴'

In [173]:
def synthetic(target_df, sample_df, type1):
    body_list=[]
    title_list=[]

    for i in range(len(target_df)):
        title = target_df.loc[i,'title']
        target = target_df.loc[i,'split_doc']
        sample = sample_df.loc[i,'split_doc']        
        if type1 == 1:
            body = target[0]+target[1]+sample[1]+sample[2]+target[2]
        if type1 == 2:
            body = target[0]+sample[0]+sample[1]+sample[2]+target[2]
        if type1 == 3:
            body = target[0] + sample[1] + sample[2]
        if type1 == 4:
            body = sample[0] + target[1] + sample[2]

        body_list.append(body)
        title_list.append(title)
    return title_list, body_list

In [163]:
def split_doc(doc):
    len_doc = len(doc)
    s,m = int(len_doc * 0.3), int(len_doc * 0.7)
    t1, t2, t3 = doc[:s], doc[s:m], doc[m:]
    target_list = [t1,t2,t3]
    return target_list

In [193]:
true_df['category'].value_counts()

7    12903
6    10955
2     8875
3     8240
1     5981
4     2887
5      145
0       11
8        3
Name: category, dtype: int64

In [257]:
df1 = true_df[true_df['category']==8].reset_index(drop=True)

In [258]:
from nltk.tokenize import sent_tokenize

df1['split_sen'] = df1['body'].apply(lambda x : sent_tokenize(x))
df1['split_doc'] = df1['split_sen'].apply(split_doc)

In [259]:
target_df = df1[:1]
sample_df = df1[-1:].reset_index(drop=True)

In [260]:
title_list, body_list = synthetic(target_df, sample_df, 1)
title_list2, body_list2 = synthetic(target_df, sample_df, 2)
title_list3, body_list3 = synthetic(target_df, sample_df, 3)
title_list4, body_list4 = synthetic(target_df, sample_df, 4)

In [261]:
title_data = title_list+title_list2+title_list3+title_list4
body_data = body_list+body_list2+body_list3+body_list4

In [262]:
df_result = pd.DataFrame({'title':title_data, 'body':body_data})

In [263]:
df_result['body'] = df_result['body'].apply(lambda x : ''.join(x))

In [264]:
df_result.head()

,title,body
0,"원자력연, 손바닥 크기 장치로 선명한 엑스선 촬영 기술 이전",한국원자력연구원은 더 선명한 영상을 촬영하면서 휴대가 가능한 다중에너지 엑스선 발생...
1,"원자력연, 손바닥 크기 장치로 선명한 엑스선 촬영 기술 이전",한국원자력연구원은 더 선명한 영상을 촬영하면서 휴대가 가능한 다중에너지 엑스선 발생...
2,"원자력연, 손바닥 크기 장치로 선명한 엑스선 촬영 기술 이전",한국원자력연구원은 더 선명한 영상을 촬영하면서 휴대가 가능한 다중에너지 엑스선 발생...
3,"원자력연, 손바닥 크기 장치로 선명한 엑스선 촬영 기술 이전",일본 정부가 후쿠시마 원전 오염수를 해양에 방류하겠다고 밝혀 우려가 커지는 가운데 ...


In [265]:
df_result.to_csv('category8.csv')

In [266]:
df = pd.read_csv('category0.csv')
df1 = pd.read_csv('category1.csv')
df2 = pd.read_csv('category2.csv')
df3 = pd.read_csv('category3.csv')
df4 = pd.read_csv('category4.csv')
df5 = pd.read_csv('category5.csv')
df6 = pd.read_csv('category6.csv')
df7 = pd.read_csv('category7.csv')
df8 = pd.read_csv('category8.csv')

In [267]:
result = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8])
len(result)

24284

In [269]:
result.head()

,Unnamed: 0,title,body
0,0,"손학규 ""文대통령, 조연 자처했지만 북미회담서 역할 못 해""",바른미래당 손학규 대표는 1일 전날 판문점에서 열린 북미 정상회동 및 남북미 정상회...
1,1,"리비아 피랍 석방 주씨 딸, 文대통령에게 편지…""벅차고 감사""",지난해 7월 리비아 무장세력에 납치됐다가 315일 만에 풀려난 한국인 주 모 씨의 ...
2,2,"민주·평화·정의, 노회찬 모욕 오세훈 일제히 비판","더불어민주당, 민주평화당, 정의당 등 여야 3당은 1일 자유한국당 소속 오세훈 전 ..."
3,3,"이총리, 돼지열병 방역현장 두번째 방문…최고수준 방역 강조",이낙연 국무총리는 5일 경기 북부 접경지역의 아프리카돼지열병 차단 방역현장을 점검했...
4,4,"이해찬 ""마스크 사재기·폭리 몰지각…코로나 사태 악용안돼""","더불어민주당 이해찬 대표는 5일 신종 코로나바이러스 감염증과 관련, ""일부 몰지각한..."


In [270]:
result.tail()

,Unnamed: 0,title,body
3999,3999,대전 외국인통합지원센터 내년 초 하나은행 대전역전점에 오픈,경기도는 카카오T 블루 택시에 대한 배차 몰아주기 의혹에 대한 실태조사에 나선다고 ...
0,0,"원자력연, 손바닥 크기 장치로 선명한 엑스선 촬영 기술 이전",한국원자력연구원은 더 선명한 영상을 촬영하면서 휴대가 가능한 다중에너지 엑스선 발생...
1,1,"원자력연, 손바닥 크기 장치로 선명한 엑스선 촬영 기술 이전",한국원자력연구원은 더 선명한 영상을 촬영하면서 휴대가 가능한 다중에너지 엑스선 발생...
2,2,"원자력연, 손바닥 크기 장치로 선명한 엑스선 촬영 기술 이전",한국원자력연구원은 더 선명한 영상을 촬영하면서 휴대가 가능한 다중에너지 엑스선 발생...
3,3,"원자력연, 손바닥 크기 장치로 선명한 엑스선 촬영 기술 이전",일본 정부가 후쿠시마 원전 오염수를 해양에 방류하겠다고 밝혀 우려가 커지는 가운데 ...


In [268]:
result.to_csv('syn_dataset.csv')